# Phase 2 – Dataset Preparation

## Dataset Overview

A total of **2000 annotated wildlife images** were used for this project.

The dataset contains bounding box annotations for the following species:

- Bear  
- Deer  
- Elephant  
- Monkey  
- Tiger  

## Dataset Split

The dataset was divided into training, validation, and testing sets as follows:

| Split       | Number of Images | Percentage |
|------------|------------------|------------|
| Training   | 1400             | 70%        |
| Validation | 400              | 20%        |
| Testing    | 200              | 10%        |
| **Total**  | **2000**         | **100%**   |

This split ensures proper model training, validation during training, and unbiased testing.

---

# Phase 3 – Baseline Model Evaluation

## Objective

Evaluate the performance of a pretrained YOLOv8n model (trained on COCO dataset) on the custom wildlife dataset before fine-tuning.

## Experimental Setup

- Model: YOLOv8n (COCO pretrained)
- Validation Images: 400
- Validation Instances: 670
- Image size: 640 × 640
- Evaluation mode: `task=detect, mode=val`

## Baseline Results

| Metric      | Value   |
|------------|----------|
| Precision  | 0.0346   |
| Recall     | 0.122    |
| mAP50      | 0.0203   |
| mAP50-95   | 0.0146   |

## Observations

- The pretrained COCO model predicted unrelated classes such as:
  - person
  - bicycle
  - car
  - airplane
- Wildlife species were frequently misclassified.
- Precision and mAP values were extremely low.
- The model failed to generalize to the wildlife domain.

## Conclusion (Baseline Phase)

The COCO-pretrained YOLOv8n model does not perform well on wildlife-specific classes due to domain mismatch and class misalignment.


---

# Phase 4 – Custom Model Training (Fine-Tuning)

## Training Configuration

- Base model: YOLOv8n
- Epochs: 50
- Image size: 640
- GPU: Tesla T4
- Training images: 1400
- Validation images: 400

## Final Validation Results (After 50 Epochs)

| Metric      | Value |
|------------|--------|
| Precision  | 0.806  |
| Recall     | 0.672  |
| mAP50      | 0.762  |
| mAP50-95   | 0.553  |

## Per-Class mAP50

| Class     | mAP50 |
|-----------|--------|
| Bear      | 0.763 |
| Deer      | 0.792 |
| Elephant  | 0.817 |
| Monkey    | 0.866 |
| Tiger     | 0.572 |

## Observations

- Significant improvement over baseline.
- The model successfully adapted to the wildlife domain.
- Monkey and Elephant achieved strong detection accuracy.
- Tiger performance is comparatively lower due to fewer training samples (class imbalance).
- Overall detection performance is suitable for deployment in tracking and counting pipeline.

---

# Conclusion

The fine-tuned YOLOv8 model demonstrates strong detection capability across the five target wildlife species.

The model is now ready for:

- Multi-object tracking integration
- Per-frame species counting
- Unique per-video counting
- Structured reporting (CSV / JSON)







---



### Installing Dependencies

In [1]:
!pip install fiftyone

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 88.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.8/112.8 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.4/112.4 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.8/74.8 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 118.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 101.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 934.5/934.5 kB 66.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 kB 18.5 MB/s eta

### Loading the dataset

In [2]:
import fiftyone as fo
import shutil
import os

import fiftyone.zoo as foz

dataset = foz.load_zoo_dataset(
    "open-images-v7",
    split="train",
    label_types=["detections"],
    classes=["Elephant", "Tiger", "Deer", "Monkey", "Bear"],
    max_samples=2000,
    dataset_name="open-images-wildlife-2000",
    drop_existing_dataset=True
)


dataset.persistent = True
dataset.save()

print(f"Dataset '{dataset.name}' saved and is now persistent.")

print(dataset)


/usr/local/lib/python3.12/dist-packages/glob2/fnmatch.py:141: SyntaxWarning: invalid escape sequence '\Z'
  return '(?ms)' + res + '\Z'


INFO:fiftyone.zoo.datasets:Downloading split 'train' to '/root/fiftyone/open-images-v7/train' if necessary


INFO:fiftyone.utils.openimages:Downloading 'https://storage.googleapis.com/openimages/2018_04/train/train-images-boxable-with-rotation.csv' to '/root/fiftyone/open-images-v7/train/metadata/image_ids.csv'


 100% |██████|    4.8Gb/4.8Gb [28.0s elapsed, 0s remaining, 195.9Mb/s]      


INFO:eta.core.utils: 100% |██████|    4.8Gb/4.8Gb [28.0s elapsed, 0s remaining, 195.9Mb/s]      


...

 100% |███████████████| 2000/2000 [7.3s elapsed, 0s remaining, 184.9 samples/s]      


INFO:eta.core.utils: 100% |███████████████| 2000/2000 [7.3s elapsed, 0s remaining, 184.9 samples/s]      


Dataset 'open-images-wildlife-2000' created


INFO:fiftyone.zoo.datasets:Dataset 'open-images-wildlife-2000' created


Dataset 'open-images-wildlife-2000' saved and is now persistent.
Name:        open-images-wildlife-2000
Media type:  image
Num samples: 2000
Persistent:  True
Tags:        []
Sample fields:
    id:               fiftyone.core.fields.ObjectIdField
    filepath:         fiftyone.core.fields.StringField
    tags:             fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:         fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    created_at:       fiftyone.core.fields.DateTimeField
    last_modified_at: fiftyone.core.fields.DateTimeField
    ground_truth:     fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)


### Filtering the Dataset

In [4]:
from fiftyone import ViewField as F

# Load original dataset
dataset = fo.load_dataset("open-images-wildlife-2000")

target_classes = ["Elephant", "Tiger", "Deer", "Monkey", "Bear"]

# Filter labels
filtered_view = dataset.filter_labels(
    "ground_truth",
    F("label").is_in(target_classes)
)

# Delete existing clean dataset if exists
if fo.dataset_exists("wildlife_clean_2000"):
    fo.delete_dataset("wildlife_clean_2000")

# Clone filtered view properly
clean_dataset = filtered_view.clone("wildlife_clean_2000")
clean_dataset.persistent = True
clean_dataset.save()

print("Clean dataset created:", clean_dataset.name)
print("Class distribution:")
print(clean_dataset.count_values("ground_truth.detections.label"))

Clean dataset created: wildlife_clean_2000
Class distribution:
{'Tiger': 428, 'Bear': 115, 'Monkey': 774, 'Deer': 1023, 'Elephant': 930}


In [10]:
# # Load dataset
# dataset = fo.load_dataset("wildlife_clean_2000")

# print("Original samples:", len(dataset))

# # Find samples with missing files
# missing_ids = []

# for sample in dataset:
#     if not os.path.exists(sample.filepath):
#         missing_ids.append(sample.id)

# print("Missing files found:", len(missing_ids))

# # Delete broken samples
# if missing_ids:
#     dataset.delete_samples(missing_ids)
#     print("Broken samples removed")

# # Save cleaned dataset
# dataset.persistent = True
# dataset.save()

# print("Remaining samples:", len(dataset))

Original samples: 2000
Missing files found: 0
Remaining samples: 2000


### Export in the yolo required format

In [12]:
# Load dataset
dataset = fo.load_dataset("wildlife_clean_2000")

export_dir = "data/dataset_yolo"

# Remove old export folder if exists
if os.path.exists(export_dir):
    shutil.rmtree(export_dir)

# Export to YOLOv5 format (YOLOv8 compatible)
dataset.export(
    export_dir=export_dir,
    dataset_type=fo.types.YOLOv5Dataset,
    label_field="ground_truth",
)

print("Export complete")
print("Total exported samples:", len(dataset))

 100% |███████████████| 2000/2000 [9.2s elapsed, 0s remaining, 123.5 samples/s]      


INFO:eta.core.utils: 100% |███████████████| 2000/2000 [9.2s elapsed, 0s remaining, 123.5 samples/s]      


Export complete
Total exported samples: 2000


In [13]:
import os
import shutil

source_images = "data/dataset_yolo/images/val"
source_labels = "data/dataset_yolo/labels/val"

flat_images = "data/flat/images"
flat_labels = "data/flat/labels"

os.makedirs(flat_images, exist_ok=True)
os.makedirs(flat_labels, exist_ok=True)

# Move images
for file in os.listdir(source_images):
    shutil.copy(os.path.join(source_images, file),
                os.path.join(flat_images, file))

# Move labels
for file in os.listdir(source_labels):
    shutil.copy(os.path.join(source_labels, file),
                os.path.join(flat_labels, file))

print("Flattened dataset created.")

Flattened dataset created.


In [14]:
import os
import random
import shutil

random.seed(42)

source_images = "data/dataset_yolo/images/val"
source_labels = "data/dataset_yolo/labels/val"

dest = "data/dataset"

splits = ["train", "val", "test"]

# Create directories
for split in splits:
    os.makedirs(f"{dest}/{split}/images", exist_ok=True)
    os.makedirs(f"{dest}/{split}/labels", exist_ok=True)

images = os.listdir(source_images)
random.shuffle(images)

train_split = int(0.7 * len(images))
val_split = int(0.9 * len(images))

train = images[:train_split]
val = images[train_split:val_split]
test = images[val_split:]

def copy_files(files, split):
    for file in files:
        shutil.copy(
            os.path.join(source_images, file),
            os.path.join(dest, split, "images", file)
        )

        label = file.replace(".jpg", ".txt")
        shutil.copy(
            os.path.join(source_labels, label),
            os.path.join(dest, split, "labels", label)
        )

copy_files(train, "train")
copy_files(val, "val")
copy_files(test, "test")

print("Dataset split completed successfully.")

Dataset split completed successfully.


### Installing YOLO

In [15]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 68.7 MB/s eta 0:00:00


In [17]:
import os

print("Train images:", len(os.listdir("/content/data/dataset/train/images")))
print("Val images:", len(os.listdir("/content/data/dataset/val/images")))
print("Test images:", len(os.listdir("/content/data/dataset/test/images")))

Train images: 1400
Val images: 400
Test images: 200


### Base Model Evaluation

In [18]:
!yolo detect val model=yolov8n.pt data=/content/data/dataset.yaml imgsz=640

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.4.14 🚀 Python-3.12.12 torch-2.10.0+cu128 CUDA:0 (Tesla T4, 14913MiB)
YOLOv8n summary (fused): 72 layers, 3,151,904 parameters, 0 gradients, 8.7 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2778.2±506.3 MB/s, size: 335.0 KB)
val: Scanning /content/data/dataset/val/labels... 400 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 400/400 2.0Kit/s 0.2s
val: New cache created: /content/data/dataset/val/labels.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 25/25 3.0it/s 8.4s
                   all        400        670     0.0346      0.122     0.0203     0.0146
                person         9

## Baseline Validation Results (Pretrained YOLOv8n)

| Metric      | Value  |
|------------|--------|
| Precision  | 0.0346 |
| Recall     | 0.122  |
| mAP50      | 0.0203 |
| mAP50-95   | 0.0146 |

## Per-Class mAP50 (COCO Predictions)

| Class       | mAP50  |
|-------------|--------|
| person      | 0.0101 |
| bicycle     | 0.0358 |
| car         | 0.00584 |
| motorcycle  | 0.0202 |
| airplane    | 0.0297 |

__________________________

### MODEL TRAINING ON CUSTOM DATASET


In [23]:
!yolo task=detect mode=train \
model=yolov8n.pt \
data=/content/data/dataset.yaml \
epochs=50 \
imgsz=640 \
batch=16 \
device=0

Ultralytics 8.4.14 🚀 Python-3.12.12 torch-2.10.0+cu128 CUDA:0 (Tesla T4, 14913MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/data/dataset.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, 

## Final Validation Results (After 50 Epochs)

| Metric      | Value |
|------------|--------|
| Precision  | 0.806  |
| Recall     | 0.672  |
| mAP50      | 0.762  |
| mAP50-95   | 0.553  |

## Per-Class mAP50

| Class     | mAP50 |
|-----------|--------|
| Bear      | 0.763 |
| Deer      | 0.792 |
| Elephant  | 0.817 |
| Monkey    | 0.866 |
| Tiger     | 0.572 |